In [1]:
import pandas as pd
import re
import collections
import pymorphy2
import nltk
nltk.download("stopwords")

from sklearn.feature_extraction.text import CountVectorizer

morph = pymorphy2.MorphAnalyzer()
stopwords = nltk.corpus.stopwords.words("russian")

[nltk_data] Downloading package stopwords to /home/bun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('Тестовое задание InnoData - Лист1.csv')

In [3]:
data['Review Title'] = data['Review Title'].fillna(' ')
data['text'] = data['Review Title']+' '+data['Review Text']

In [4]:
def to_norm(text):
    clean_text = re.sub('[,.!?a-z0-9-]', '', text.lower()).split()
    for ind, word in enumerate(clean_text):
        try:
            if word in ['не', 'нет']: #and morph.parse(clean_text[ind+1])[0].tag.POS in {'VERB', 'NOUN'}:
                clean_text[ind] = word+'_'+clean_text[ind+1]
                del clean_text[ind+1]
        except IndexError:
            continue
            
    norm_text = [morph.parse(w)[0].normal_form for w in clean_text]
    import_text = [w for w in norm_text if w not in stopwords]
    if len(import_text)<0:
        return 'отзыв_не_обработан'
    else:
        return ' '.join(import_text)

In [5]:
all_reviews = data.dropna(axis=0, how='any')
all_reviews['norm_text'] = all_reviews['text'].apply(to_norm)

all_reviews = all_reviews.reset_index(drop=True)

/home/bun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
cv = CountVectorizer(ngram_range=(2, 2),max_df=0.95, min_df=0.01, max_features=5000, stop_words = ['отзыв_не_обработан'])
freq = cv.fit_transform(all_reviews['norm_text']).toarray()
voc = cv.get_feature_names()
review_token = pd.concat([all_reviews[['type', 'Star Rating', 'text']], pd.DataFrame(freq, columns=voc)], axis=1)

In [11]:
star_tokens = dict()

for name, group in review_token.groupby('Star Rating'):
    tokens = group.iloc[:,3:].sum().sort_values(ascending=False)[:20]#.reset_index().iloc[:10,0].values
    star_tokens[name] = tokens 

### Лексические характеристики для каждой группы отзывов, по количеству звезд

In [13]:
for i in star_tokens:
    print('Словосочетания чаще всего встречающиеся в отзывах с {} звездами'.format(i))
    print(star_tokens[i])
    print('---------------------------------------------------------------')

Словосочетания чаще всего встречающиеся в отзывах с 1 звездами
личный кабинет            357
купить билет              120
логин пароль              115
последний обновление       90
вход личный                74
перестать работать         74
войти личный               67
приложение не_работать     61
не_мочь войти              60
это приложение             59
постоянно вылетать         50
не_мочь зайти              45
приложение постоянно       45
дата рождение              44
обновление приложение      42
зайти личный               40
покупка билет              38
приложение вылетать        36
весь равно                 36
регистрация рейс           36
dtype: int64
---------------------------------------------------------------
Словосочетания чаще всего встречающиеся в отзывах с 2 звездами
личный кабинет             127
купить билет                59
логин пароль                53
последний обновление        37
покупка билет               35
вход личный                 29
обновление 

В результате последнего обновления многие пользователи жалуются на нестабильную работу приложения, 
проблемы с входом в личный кабинет, а так же на доступ к приложению по отпечатку пальца.
А точнее, 
 - после обновления сбрасываются личные данные пользователи, и их нужно вводить заново
 - вход по отпечатку не работает или постоянно предлагает перейти на вход только  по отпечатку, 
   указав больше не спрашивать, но на следующий раз все повторяется снова

Интересно, что даже в отзывах с высокой оценкой пользователи упоминают проблемы с доступом к приложению. 
Чаще всего здесь имеется в виду, невозможность зайти в личный кабинет, 
так же упоминается недоступность некоторых функций приложения, покупка билетов и прочее.

In [221]:
review_token.groupby('type')['text'].count()

type
1    3982
2    2000
Name: text, dtype: int64

In [190]:
def find_ngrams(input_list, n):
    return [' '.join(words) for words in zip(*[input_list[i:] for i in range(n)])]

def add_to_dict(diction, key, value):
    if key in diction.keys():
        diction[key].extend(value)
    else:
        diction[key] = value

In [196]:
find_around = dict()

for ind, row in all_reviews.iterrows():
    text = row['norm_text'].split()
    text.extend(find_ngrams(text, 2))
    for i, w in enumerate(text):
        if w in star_tokens[1]:
            try:
                around = text[i-2:i+2]
                del around[2]
            except IndexError:
                pass
            add_to_dict(find_around, w, around)